In [1]:
import numpy as np

In [2]:
import matplotlib.pyplot as plt

In [3]:
import numpy as np
from scipy import optimize

def compute_utilities(y1, y2, prob, s_1, s_2, theta_x):
    p_0, p_1, p_2, q_0, q_1, q_2 = prob
    
    # Low type utilities
    u_0_l = 0
    u_1_l = theta_x * ((1 - q_2 - q_0) * 0 + (q_2 + q_0) * y1) + \
            (1 - theta_x) * ((1 - p_2 - p_0) * y1/2 + (p_2 + p_0) * y1) - s_1
    
    u_2_l = theta_x * ((1 - q_1 - q_0) * 0 + (q_1 + q_0) * y2) + \
            (1 - theta_x) * ((1 - p_1 - p_0) * y2/2 + (p_1 + p_0) * y2) - s_2
    
    # Both firms utility for low type depends on which value is higher
    if y1 > y2:
        u_b_l = (theta_x * (q_0 * y1 + q_1 * y2 + q_2 * y1 + (1 - q_0 - q_1 - q_2) * 0) +
                (1 - theta_x) * (p_0 * y1 + p_1 * (y1/2 + y2/2) + p_2 * y1 + 
                               (1 - p_0 - p_1 - p_2) * (y1/2 + y2/4))) - s_1 - s_2
    else:
        u_b_l = (theta_x * (q_0 * y2 + q_1 * y2 + q_2 * y1 + (1 - q_0 - q_1 - q_2) * 0) +
                (1 - theta_x) * (p_0 * y2 + p_1 * y2 + p_2 * (y1/2 + y2/2) + 
                               (1 - p_0 - p_1 - p_2) * (y2/2 + y1/4))) - s_1 - s_2
    
    # High type utilities
    u_0_h = 0
    u_1_h = theta_x * ((1 - q_2 - q_0) * y1/2 + (q_2 + q_0) * y1) + \
            (1 - theta_x) * y1 - s_1
    
    u_2_h = theta_x * ((1 - q_1 - q_0) * y2/2 + (q_1 + q_0) * y2) + \
            (1 - theta_x) * y2 - s_2
    
    # Both firms utility for high type
    if y1 > y2:
        u_b_h = (theta_x * (q_0 * y1 + q_1 * (y1/2 + y2/2) + q_2 * y1 + 
                          (1 - q_0 - q_1 - q_2) * (y1/2 + y2/4)) +
                (1 - theta_x) * y1) - s_1 - s_2
    else:
        u_b_h = (theta_x * (q_0 * y2 + q_1 * (y2/2 + y1/2) + q_2 * y1 + 
                          (1 - q_0 - q_1 - q_2) * (y2/2 + y1/4)) +
                (1 - theta_x) * y2) - s_1 - s_2
    
    return [u_0_l, u_1_l, u_2_l, u_b_l], [u_0_h, u_1_h, u_2_h, u_b_h]

def find_optimal_choice(y1, y2, prob, s_1, s_2, theta_x):
    utils_l, utils_h = compute_utilities(y1, y2, prob, s_1, s_2, theta_x)
    choice_l = np.argmax(utils_l)
    choice_h = np.argmax(utils_h)
    return choice_l, choice_h

def compute_equilibrium(prob, s_1, s_2, theta_x, n_samples=1000):
    # Sample points uniformly in [0,1] x [0,1]
    np.random.seed(42)  # For reproducibility
    y1_samples = np.random.uniform(0, 1, n_samples)
    y2_samples = np.random.uniform(0, 1, n_samples)
    
    # Initialize counters for each choice
    low_choices = np.zeros(4)  # [0, 1, 2, both]
    high_choices = np.zeros(4)
    
    # Count choices for y1 > y2 and y1 ≤ y2 separately
    y1_greater = np.zeros(4)
    y1_lesser = np.zeros(4)
    
    for y1, y2 in zip(y1_samples, y2_samples):
        choice_l, choice_h = find_optimal_choice(y1, y2, prob, s_1, s_2, theta_x)
        low_choices[choice_l] += 1
        high_choices[choice_h] += 1
        
        # Track choices conditional on y1 > y2 or y1 ≤ y2
        if y1 > y2:
            if choice_h == 3:  # if chose both
                y1_greater[choice_h] += 1
        else:
            if choice_h == 3:  # if chose both
                y1_lesser[choice_h] += 1
    
    # Normalize to get probabilities
    p_0, p_1, p_2, p_b = low_choices / n_samples
    q_0, q_1, q_2, q_b = high_choices / n_samples
    
    # Calculate h_1, h_2 (probability of being high type conditional on applying)
    h_1 = (theta_x * (q_1 + q_b)) / (theta_x * (q_1 + q_b) + (1 - theta_x) * (p_1 + p_b))
    h_2 = (theta_x * (q_2 + q_b)) / (theta_x * (q_2 + q_b) + (1 - theta_x) * (p_2 + p_b))
    
    # Calculate f_h (probability of preferring firm 1 being high type conditional on applying to both)
    total_both_choices = y1_greater[3] + y1_lesser[3]
    f_h = y1_greater[3] / total_both_choices if total_both_choices > 0 else 0
    
    return [p_0, p_1, p_2, q_0, q_1, q_2], h_1, h_2, f_h

def get_profits(theta_x, s_1, s_2, c, prob_init=[1,0,0,1,0,0], n_samples=1000):
    # Find fixed point of probabilities
    prob = prob_init
    max_iter = 100
    convergence_threshold = 1e-6
    
    for i in range(max_iter):
        prob_n, h_1, h_2, f_h = compute_equilibrium(prob, s_1, s_2, theta_x, n_samples=n_samples)
        distance = np.linalg.norm(np.array(prob_n) - np.array(prob))
        
        if distance < convergence_threshold:
            break
            
        prob = prob_n
        
    # Get final equilibrium values
    prob_n, h_1, h_2, f_h = compute_equilibrium(prob, s_1, s_2, theta_x, n_samples=n_samples)
    p_0, p_1, p_2, q_0, q_1, q_2 = prob_n
    
    # Compute profits using equilibrium values
    p_b = 1 - p_0 - p_1 - p_2
    q_b = 1 - q_0 - q_1 - q_2
    
    pi_1 = (h_1 ** 2 * (q_1 ** 2 * (1 - 2*c) + 
                        2 * q_1 * (q_0 + q_2) * (1 - c) + 
                        q_b ** 2 * (1 - (1 - f_h)**2 - 2 * c) + 
                        2 * q_b * (q_0 + q_2) * (f_h - c) + 
                        2 * q_b * q_1 * (f_h/2 + 1/2 - 2*c)) +
            2 * h_1 * (1-h_1) * (q_1 * (p_1 + p_b) * (1 - 2*c) + 
                                q_1 * (p_0 + p_2) * (1 - c) + 
                                q_b * (p_1 + p_b) * (f_h - 2*c) + 
                                q_b * (p_0 + p_2) * (f_h - c)) +
            (1 - h_1) ** 2 * ((p_1 + p_b) ** 2 * (-2*c) + 
                             2 * (p_1 + p_b) * (p_0 + p_2) * (-c)))
    
    pi_2 = (h_2 ** 2 * (q_2 ** 2 * (1 - 2*c) + 
                        2 * q_2 * (q_0 + q_1) * (1 - c) + 
                        q_b ** 2 * (1 - f_h**2 - 2 * c) + 
                        2 * q_b * (q_0 + q_1) * (1 - f_h - c) + 
                        2 * q_b * q_2 * ((1 - f_h)/2 + 1/2 - 2*c)) +
            2 * h_2 * (1-h_2) * (q_2 * (p_2 + p_b) * (1 - 2*c) + 
                                q_2 * (p_0 + p_1) * (1 - c) + 
                                q_b * (p_2 + p_b) * (1 - f_h - 2*c) + 
                                q_b * (p_0 + p_1) * (1 - f_h - c)) +
            (1 - h_2) ** 2 * ((p_2 + p_b) ** 2 * (-2*c) + 
                             2 * (p_2 + p_b) * (p_0 + p_1) * (-c)))
    
    return pi_1, pi_2

def get_reaction(s_2, theta_x, c, n_samples=1000):
    """Find optimal reaction of firm 1 given firm 2's strategy"""
    result = optimize.minimize(
        lambda signal: -get_profits(theta_x, signal[0], s_2, c, n_samples = n_samples)[0],
        x0=[s_2],
        method='Nelder-Mead',
        bounds=[(0, 1)],
        options={
            'maxiter': 1000,
            'disp': False,
            'adaptive': True
        }
    )
    return result.x[0]

In [4]:
c = 0.2

In [5]:
theta_x = 0.2

In [6]:
s_2 = 0.2

In [7]:
get_reaction(s_2, theta_x, c, n_samples=100)

0.22000000000000003

In [8]:
get_reaction(s_2, theta_x, c, n_samples=1000)

0.2924999999999999

In [9]:
get_reaction(s_2, theta_x, c, n_samples=10000)

0.3074999999999999

In [11]:
get_reaction(s_2, theta_x, c, n_samples=100000)

0.2987109374999999

In [10]:
error

NameError: name 'error' is not defined

In [ ]:
s2_grid = np.linspace(0.01,0.99,100)

In [ ]:
c = 0.2

In [ ]:
theta_x = 0.2

In [ ]:
get_reaction(0.5, theta_x, c)

In [ ]:
reactions = [*map(lambda s2: get_reaction(s2, theta_x, c), s2_grid)]

In [ ]:
plt.plot(s2_grid, s2_grid)
plt.plot(s2_grid, reactions)